Dans ce dataset: https://raw.githubusercontent.com/fspot/INFMDI-721/master/lesson5/products.csv, chaque ligne correspond à un produit alimentaire mis en vente par un utilisateur.

Objectif: cleaner le dataset.

On aimerait avoir une colonne de prix unifiés en euros. Problème: la currency n'est pas indiquée pour tous les produits: il va falloir essayer de "deviner" les currency manquantes, en se basant sur l'adresse IP de l'utilisateur. La colonne "infos" liste des ingrédients présents dans le produit. On préfèrerait avoir une colonne de type bool par ingrédient, indiquant si le produit contient ou non cet ingrédient. Voic une liste d'APIs qui peut vous être utile :

https://github.com/public-apis/public-apis (mais vous pouvez en utiliser d'autres si vous le voulez).


In [58]:
import pandas as pd
import requests
import ipaddress

In [59]:
products = pd.read_csv(
    'https://raw.githubusercontent.com/fspot/INFMDI-721/master/lesson5/products.csv',sep=';')

In [60]:
# Cleaning the price
price = products["price"].str.split(" ", n = 1, expand = True) 
products.price = price[0]
products["currency"] = price[1]
products.head(20)

,username,ip_address,product,price,infos,currency
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar,None
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,None
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,None
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,None
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",None
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33,Ingredients: sugar and milk,MGA
6,afairholme6,127.197.204.119,Chicken - Tenderloin,484.83,May contain sugar,None
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar,None
8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish",None
9,tkendrew9,22.32.234.215,Pop - Club Soda Can,861.25,"May contain peanut, sugar, milk and fish",None


In [61]:
import ipaddress
def is_ipv4(string):
    try:
        ipaddress.IPv4Network(string)
        return True
    except ValueError:
        return False

In [62]:
products["ip_address"].head(20)

0     666.666.666.666
1                nope
2      240.177.79.234
3       26.191.237.49
4       58.90.204.239
5        226.52.32.70
6     127.197.204.119
7       189.169.17.54
8     187.129.113.105
9       22.32.234.215
10     75.254.207.163
11      17.105.113.72
12     208.239.186.79
13       116.18.56.20
14      87.198.88.207
15      138.44.156.99
16      18.61.195.234
17     150.214.20.140
18    157.197.190.113
19      219.75.213.15
Name: ip_address, dtype: object

In [68]:
restcountries_data = requests.get('https://restcountries.eu/rest/v2/all').json()

In [74]:
currencies_to_eur = requests.get(
    'http://data.fixer.io/api/latest?access_key=5227d9e628debf99cabeea740b178492'
).json()


In [75]:
def country_ip(ip):
    url = f"https://api.ip2country.info/ip?{ip}"
    data = requests.get(url).json()
    return data['countryCode']
country_ip("157.145.5.133")

'US'

In [76]:
countrycode_to_currency = {
    country['alpha2Code']: country['currencies'][0]['code']
    for country in restcountries_data
}

countrycode_to_currency = {}
for country in restcountries_data:
    country_code = country['alpha2Code']
    currency = country['currencies'][0]['code']
    countrycode_to_currency[country_code] = currency

In [79]:
price = splitted.str[0].astype(float)
currency = splitted.str[1]

products['price'] = price
products['currency'] = currency

products = products[products.currency.notna()].reset_index()
products['price_eur'] = products.price * products.currency.map(currencies_to_eur)